In [67]:
import tensorflow as tf
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem

from sklearn.model_selection import train_test_split

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, \
    Activation, BatchNormalization, LSTM, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [69]:
df = pd.read_csv('cmpd.csv')
count = 0

train = pd.DataFrame(columns=['inchikey', 'smiles', 'activity'])
test = pd.DataFrame(columns=['inchikey', 'smiles', 'activity'])

for idx in range(len(df)):
    if df.iloc[idx, -2] == 'train':
        train.loc[idx] = df.iloc[idx, [0, 1, 3]]
        count += 1
    else:
        test.loc[idx] = df.iloc[idx, [0, 1, 3]]
        count += 1

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [70]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1].to_list()
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1].to_list()

In [71]:
def y_to_vec(list):
    for idx in range(len(list)):
        if list[idx] == 'active':
            list[idx] = 1
        else:
            list[idx] = 0
    return list

y_train = y_to_vec(y_train)
y_test = y_to_vec(y_test)

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [72]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.9, random_state=23)

print(len(X_train)) # 3181
print(len(X_val))   # 796
print(len(X_test))  # 1553

print(len(y_train)) # 3181
print(len(y_val))   # 796
print(len(y_test))  # 1553

3579
398
1553
3579
398
1553


In [73]:
def make_dataset(df):
    temp_list = []

    for idx in range(len(df)):
        temp_list.append(df.iloc[idx, :])
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
    tokenizer.fit_on_texts(temp_list)
    top_k = len(tokenizer.word_index)

    temp_seq = tokenizer.texts_to_sequences(temp_list)
    temp_vec = tf.keras.preprocessing.sequence.pad_sequences(temp_seq, padding='post')
    return temp_vec

X_train = make_dataset(X_train)
X_val = make_dataset(X_val)
X_test = make_dataset(X_test)

In [74]:
print(X_train.shape)
print(len(np.unique(X_train)))

(3579, 2)
7121


In [81]:
model = Sequential()
model.add(Embedding(input_dim=len(np.unique(X_train)), output_dim=10, input_length=2))
model.add(Flatten())
model.add(Dense(8))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(8))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [83]:

model.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics='acc'
)

model.fit(
    X_train, y_train,
    validation_data=[X_val, y_val],
    epochs=100, batch_size=64
)

model.save(filepath='./model.h5')

Epoch 1/100
56/56 [==============================] - 1s 12ms/step - loss: 0.7434 - acc: 0.4951 - val_loss: 0.6954 - val_acc: 0.4749
Epoch 2/100
56/56 [==============================] - 1s 10ms/step - loss: 0.6390 - acc: 0.6594 - val_loss: 0.6932 - val_acc: 0.5201
Epoch 3/100
56/56 [==============================] - 1s 9ms/step - loss: 0.5503 - acc: 0.7625 - val_loss: 0.6933 - val_acc: 0.5327
Epoch 4/100
56/56 [==============================] - 1s 9ms/step - loss: 0.4569 - acc: 0.8220 - val_loss: 0.7282 - val_acc: 0.5126
Epoch 5/100
56/56 [==============================] - 1s 9ms/step - loss: 0.3471 - acc: 0.8958 - val_loss: 0.7747 - val_acc: 0.5000
Epoch 6/100
56/56 [==============================] - 1s 9ms/step - loss: 0.2666 - acc: 0.9290 - val_loss: 0.8988 - val_acc: 0.5201
Epoch 7/100
56/56 [==============================] - 1s 9ms/step - loss: 0.2104 - acc: 0.9497 - val_loss: 0.9887 - val_acc: 0.5000
Epoch 8/100
56/56 [==============================] - 1s 9ms/step - loss: 0.1627 -

In [78]:

loss, acc = model.evaluate(
    X_test, y_test
)


y_pred = model.predict(y_test)[:, 1]
for idx in range(len(y_pred)):
    if y_pred[idx] > 0.5:
        y_pred[idx] = 1.
    else:
        y_pred[idx] = 0.
_y_test = np.max(y_test, axis=1)

print("y_pred : ", y_pred)
print("y_test : ", _y_test)

acc_score = accuracy_score(_y_test, y_pred)
recall = recall_score(_y_test, y_pred)
precision = precision_score(_y_test, y_pred)
f1 = f1_score(_y_test, y_pred)

print("loss : ", loss)
print("acc : ", acc)
print("acc_score : ", acc_score)
print("recall : ", recall)
print("precision : ", precision)
print("f1 : ", f1)
# log_loss = metrics.log_loss(np.max(y_test, axis=1), y_pred, labels=[0, 1])
# print(log_loss)

49/49 [==============================] - 0s 3ms/step - loss: 1.5116 - acc: 0.6362
y_pred :  [0. 0. 0. ... 0. 0. 0.]
y_test :  [1. 1. 1. ... 1. 1. 1.]
loss :  1.511612057685852
acc :  0.6361880302429199
acc_score :  0.0
recall :  0.0
precision :  0.0
f1 :  0.0


C:\Users\fourlab\anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
# precision, recall, _ = metrics.precision_recall_curve(np.max(y_test, axis=1), y_pred, pos_label=None)
# fpr_roc, tpr_roc, _ = metrics.roc_curve(np.max(y_test, axis=1), y_pred, pos_label=None)

# auc1 = metrics.auc(recall, precision)
# auc2 = metrics.auc(fpr_roc, tpr_roc)

# print(auc1)
# print(auc2)

In [80]:
# train val 0.9 (BatchNormal)
# 5.406352996826172 0.5251256227493286

# train val 0.9 (Dropout)
# 31.515090942382812 0.5175879597663879

# train val 0.9 (BatchNormal + Dropout)
# 8.611763000488281 0.5150753855705261

# train val 0.9 (BatchNormal + adjust node)
# 3.570016622543335 0.5025125741958618